CompoundCalulator 20200816
==========================
Given a list of base compound names and molecular weights, this notebook calculates possible derivatives (e.g. metabolites) in 2 phases, and adds adducts and losses resulting in a list of possible ion masses with labels.
The intention is that this be used with the PCVG compound finding code, and the idea is that once we have a suggested identification, e.g. ibuprofen, we can calculate other potential masses and look for matches with the ions we've found. This eliminates a lot of manual calculations and allows interactive investigation, e.g. if we find an adduct that corresponds to addition of C2H3O2Na we can quickly incorporate it into our lists and see if it shows up elsewhere. It also allows us to look for forms that we might not otherwise consider.

To do
----
- Add UI - a form?
- Clean up and standardize code
- Refactor the Composition class to use Count and include adduct, etc.


Changes
-------

20201007

All user-defined parameters accessible in "Setup"...code can be executed with "Run selected cell and all below"
20201005

Modifications are separated from the base compound with underscores so that only losses have a minus sign making the putput easier to read.

The base compound information is now a list of (name, mass) tuples which provides a way to incorporate specific known modifications. For example, vinpocetin shows a loss of C2H4, possibly following oxidations, which then undergoes further pahase 1 and 2 modifications so the base_compound list contains both Vinpo and Apo-vinpo.

The output can now include hereodimers which are calculated by generating new compounds from all pairs of modified compounds, i.e. after the phase 1 and phase 2 nodifications have been generated but prior to generating the final list with adducts. NOTE: this can generate a lot of extra entries!

In [1]:
from itertools import combinations
from itertools import groupby

Class and function definitions
------------------------------
The basic entity is a 'Composition'...NB. this is not an elemental composition but simply a text label, a count and a mass. When compositions are combined, the labels are concatenated (using a specified separator character) and the masses are added.

In [2]:
from dataclasses import dataclass

@dataclass
class Composition:
    Name: str = ""
    Count: int = 1
    Mass: float=-1
    Root:  str = ""       #the composition this on is based on 
    
    mods = {}
    
    def __init__(self, name, count, mass=None, root=None):
        self.Name = f'{name}' if count == 1 else f'{count}({name})'
        self.Count = 1    # there's only one of these even if the 'count' (really a multiplier) is gretaer
        self.Mass = mass if mass else self.Mods[name]*count        
        
        if root:
            self.Root = root
        else:
            self.Root = name
    
    # Make the Composition from a (Name, Count) tuple
    @classmethod
    def from_tuple(cls, t):
        return Composition(t[0],t[1])

    # make a composition from a list of (Name,Count)tuples
    @classmethod
    def from_tuple_list(cls, t_list):
        comp = None
        
        for t in t_list:           
            if not comp:               
                comp = Composition.from_tuple(t)    #create a comp from the first in the list so we can append others to it
            else:
                comp2 = Composition.from_tuple(t)
                comp = comp.add_comp(comp2, sep='.')
                
        return comp
    
    @classmethod
    def proton(cls):
        return Composition('H+', 1, 1.00727)
    
    def protonate(self):
        return self.add_comp(Composition('H+', 1, 1.00727), sep='.')
    
    def deprotonate(self):
        return self.add_comp(Composition('[-H+]-', 1, -1.00727), sep='.')

    def make_copy(self, mult=1):
        return Composition(self.Name, self.Count*mult, self.Mass*mult, self.Root)
    
    def label(self):
        return self.Name
    
    # Merge two compositions to generate a new one with a mass
    def add_comp(self, comp1, sep='_'):
        new_name = self.label() + sep + comp1.label()
        new_mass = self.Mass + comp1.Mass
        return Composition( new_name, 1, root=self.Root, mass=new_mass)

    
    # expands a composition by generating a list of Name repeated Count times
    def expand(self):
        result = [self.Name for n in range(self.Count)]
        return result

#provide the set of allowed changes
Composition.Mods = {'OH':15.99492,
        'COOH':29.97418,     #COOH is CH3->COOH, i.e. +O2, -H2)
        'gluc':176.032088,
        'sulphate':79.956815,
        'NH3':17.026549,     #adducts from here
        'Na-H':21.981944,
        'K-H':37.955881,
        'K*H': 39.9540,
        'Ca-2H': 37.946941,
        'H2O':-18.010565,
        'NaAc': 82.003074,
        'NaFo': 67.987424,     # sodium formate
        'KFo': 83.961361,
        'C2H4O2':60.021129,
        'CH2O2':46.005479,
        'CHO2': 44.997654,
        'CO2':-43.989829,
        'Am':-17.026549}
 
print('Proton: ', Composition.proton())

a = Composition('Na-H',2)
print('Normal init:', a)

b = Composition.from_tuple(('K-H',2))
print('From tuple:', b)

ab = a.add_comp(b, sep='.')
print('From merge:', ab)

print('Expanded:', a.expand())

t_list = [('Na-H',2),('K-H',2), ('NH3', 1)]
abc = Composition.from_tuple_list(t_list)
print('From tuple list:', abc)

Proton:  Composition(Name='H+', Count=1, Mass=1.00727, Root='H+')
Normal init: Composition(Name='2(Na-H)', Count=1, Mass=43.963888, Root='Na-H')
From tuple: Composition(Name='2(K-H)', Count=1, Mass=75.911762, Root='K-H')
From merge: Composition(Name='2(Na-H).2(K-H)', Count=1, Mass=119.87565, Root='Na-H')
Expanded: ['2(Na-H)']
From tuple list: Composition(Name='2(Na-H).2(K-H).NH3', Count=1, Mass=136.902199, Root='Na-H')


In [25]:
def old_make_combinations(limit_list, max_combinations):
    """Given a list of limits as tuples (comp, upper_limit), return all combinations to a given maximum value
    """

    entities = []

    # Use the limit_list to generate an expanded list of individual entities, i.e. [('X', 2), ('Y',2)] +> X, X, Y, Y
    for l in limit_list:
        for n in range(l[1]):
            entities.append(l[0])

    entity_combinations = []

    # Now find all combinations of 1 entity, 2 entities...to the max number required
    # This will include duplicates, e.g. x,y and y,x
    for i in range(1, max_combinations + 1): 
        entity_combinations += list(combinations(entities, i))

    # making this into a set will find the unique combinations.
    # initially each combination tuple was sorted to make sure they were canonicalized, but this doesn't seem to be needed
    # i.e. combs = [tuple(sorted(c)) for c in entity_combinations]; cel = set(combs)
    
    csl=set(entity_combinations)   

    csa = []

    # we convert these back into the form ('x',2)('y,'1) by grouping the elements of each combination
    # and recording the element and its count...Note each group has to be converted to a list for this to work
    for c in csl:
        csa.append( [(key, len(list(group))) for key, group in groupby(c)])
    
    return csa
        
combs = old_make_combinations([('y',3), ('x',2), ('z',2)], 3)
print(len(combs),'should be 17')   #...should be 17

17 should be 17


In [4]:
def add_mods(compounds, limits, sep='_'):
    """
    Adds modifications to each compound in the list returning the new compound list.
    The modfications are provided as a list of (mods, max count) tuples
    """
    mods = []

    # Make the compounds by copying the base and adding the possible mods
    for c in compounds:
        for l in limits:
            for i in range(l[1]):
                new_comp = c.make_copy().add_comp(Composition(l[0], i+1), sep=sep)
                mods.append(new_comp)
                #print(new_comp)

    compounds += mods
    
    return compounds


Setup
-----

Provide the  base compound information and other parameters.
The base compounds are supplied as a list of (name, mass) tuples.
As shown below, the mass need not be a real known compound but can be an observed and unexplained peak so that it's potential derivatives are generated.

All user-defined parameters are set here so, once they are set, the code can be executed with 'Run selected cell and all below"

In [7]:
import os

# Define the compound(s) we want to work with
#
base_compounds = [('Guan', 283.091669),  # Guanosine
#                   ('F', 151.0489)       # fragment-H+
#                   ('x678', 678.5042),    # unknown at 679.5115 - H+
#                   ('Form', 46.005479),   # formic acid
#                   ('Et3N', 101.120449)
                 ]  # tri-ethyl ammonia   

#base_compounds = [('Ibu', 206.1307)]   #Identifier + MW
#base_compounds = [('Vinpo', 350.1994),('Apo_vinpo', 322.1681)]  #Apo_vinpo is vinpo-C2H4
#base_compounds = [('x543', 543.2068+1.00727)] 

multimer_limit = 3              # maximum multimer count
max_adduct_count = 16             # total number of adducts allowed
ionization = 'positive'          # only 'negative' changes the settings...anything else is 'positive'
include_hetero_dimers = True     # if True, calculate dimers of *different* compounds

# Output parameters
# the default file name is based on the first compound name and the polarity selected
base_name = base_compounds[0][0]  #Extract the name of the first compound to use for file naming

output_mass_limit = 1000    # masses greater than this are not written to the file
xic_width = 0.0             # if 0 the normal output form is used...alternative, e.g. 0.01, to generate the PeakView compatible form

save_ion_list = False
write_locally = False       # write to the same location as the notebook (useful for Colab); if 'False' a file path is required

# define file path (used if write_locally = False)
# this is a convenient platform independent way to provide file paths as a list of directories
f_dir = os.sep + os.path.join('Users','ronbonner','Data','PCA')

# Define the limits for metabolites and adducts...
# Defining this way is not required but allows metabolite and adduct sets to be easily changed depending on polarity.

phase1_limits = [('OH', 0), ('COOH', 0)]  # metabolite modifications - phase 1

if ionization == 'negative':
    phase2_limits = [('gluc', 1)] #, ('sulphate', 1)]
    adduct_limits = [('Na-H', 3), ('K-H', 2), ('NaAc',2), ('NaFo', 1)]  
    loss_limits = [('H2O',2), ('CO2',1)]
else:
    phase2_limits = [('gluc', 0)]
    adduct_limits = [('Na-H', 6), ('K-H', 6), ('K*H', 4), ('NaFo', 0), ('KFo',0), ('CH2O2',10)]
    loss_limits = [('H2O',0), ('Am', 0)]

Step 1 - Adduct generation
---------------------------

Generate a list of possible adduct forms by generating all comibnations of adducts (up to the specified limit) and selecting the unique forms (i.e. as far as we are concerned, a+b+a is the same as a+a+b). Note: this approach would also work if we wanted to allow combinations of the metabolites. These will be added to each compound.

In [8]:
adduct_combs = make_combinations(adduct_limits, max_adduct_count)
    
adduct_comps = [Composition.from_tuple_list(c) for c in adduct_combs]
adduct_comps = sorted(adduct_comps, key=lambda x: x.Mass)

print(len(adduct_comps),'adduct forms')

# for ac in adduct_comps:
#     print(ac)

2079 adduct forms


Step 2 - Compound generation
-----------------------------

We convert the base compound list to a of compositions and then successively apply the various modifications, generating extended compound lists, as follows

- phase 1
- phase 2

Then calculate the dimers and heterodimers (if desired)

In [9]:
# Make the compounds by copying the base and adding the possible mods

compounds = [Composition(name, 1, mass) for name, mass in base_compounds]
        
compounds = add_mods(compounds, phase1_limits)
print(len(compounds), 'compounds after phase 1')

compounds = add_mods(compounds, phase2_limits)
print(len(compounds), 'after phase 2')

multimers = []

for c in compounds:
    for m in range(2, multimer_limit+1):
        new_comp = c.make_copy(m)
        multimers.append(new_comp)

if include_hetero_dimers:
    for i, c in enumerate(compounds):
        for j in range(i+1, len(compounds)):
            new_comp = c.make_copy()
            new_comp_2 = compounds[j].make_copy()
            new_comp = new_comp.add_comp(new_comp_2, sep='+')
            multimers.append(new_comp)
    
compounds += multimers

print(len(compounds), 'with multimers')

compounds = add_mods(compounds, loss_limits, sep='-')
print (len(compounds), 'after losses')

# for c in compounds:
#     print(c)

1 compounds after phase 1
1 after phase 2
3 with multimers
3 after losses


Step 3 - Generate ion forms
----------------------------

We now add all the adduct forms to each of the compounds. The approach relies on adducts being formed by replacing labile protons and are therefore indpendent of the polarity; the final form is determined by providing a charge agent, i.e. adding or subtracting protons.


In [10]:
ion_forms = []  

for c in compounds:
    
    # add the base compound, with a proton added or subtracted deopending on the ionization mode
    new_comp = c.make_copy()
    if ionization == 'negative':
        new_comp = new_comp.deprotonate()
    else:
        new_comp = new_comp.protonate() 
        
    ion_forms.append(new_comp)   
    
    # then add the adduct forms
    for a in adduct_comps:
        new_comp = c.make_copy().add_comp(a, sep='.')
        if ionization == 'negative':
            new_comp = new_comp.deprotonate()
        else:
            new_comp = new_comp.protonate()
        ion_forms.append(new_comp)       
        
print(len(ion_forms), 'ion forms')

# for ion in sorted(ion_forms, key=lambda x: x.Mass):
#     if 495 < ion.Mass < 500:
#         print(ion)

6240 ion forms


Step 4 - Save the mass/name list
--------------------------------

Optionally save the ion forms as a simple tab delimited text file.
- the main format is: mass, label
- an additional format: mass, xic width, name is intended to be used with PeakView Extract XIC (by importing it)

The list can also be truncated to an upper mass limit.

To be sure the file exists, we re-open it and count the nuber of lines

In [11]:
if save_ion_list:

    line_count = 0

    ion_forms = sorted(ion_forms, key=lambda x: x.Mass)

    if ionization == "negative":
        f_name = f'{base_name} ions neg.txt'
    else:
        f_name = f'{base_name} ions pos.txt'

    output_path = f_name if write_locally else os.path.join(f_dir, f_name)

    print (output_path)

    with open(output_path, 'w') as f:

        for ion in ion_forms:

            if ion.Mass > output_mass_limit: 
                break       

            if xic_width:
                f.write('{:10.4f}\t{}\t{}\n'.format(ion.Mass, xic_width,ion.Name))
            else:
                f.write(f'{ion.Mass:10.4f}\t{ion.Name}\n')

            line_count += 1

        f.close()

    print(line_count, 'lines written to', output_path)
    
    with open(output_path, 'r') as f:    
        lines_read = f.readlines()    
        f.close()

    print(len(lines_read), 'read')
else:
    print('Ion list not saved')

Ion list not saved


# Step 5 - Matching

Compare the list we generated to an input peak list (assumed to be mass and intesity)

## 5.1 Imports and function definitions

In [12]:
def get_match_stats(matches, peaks, tic):

    matched_indices = set([m[0] for m in matches])   # get the unique indices since a peak may have more than one match

    matched_inten = sum([peaks[i][1] for i in matched_indices])  # sum the intensities

    percent_matched = matched_inten * 100/tic
    
    return matched_indices, percent_matched

In [13]:
def read_peak_list(peak_file_path):
    
    peaks = []    #list of (mass, inten) tuples

    with open(peak_file_path, 'r') as f:  
    
        for line in f:
            parts = line.split()
            peaks.append((float(parts[0]), float(parts[1])))  

        f.close()
    
    peaks = sorted(peaks, key = lambda x: x[0])     # ensure the list is sorted by mass

    masses, intens = zip(*peaks)

    return peaks, sum(intens), max(intens)    # peaks, tic, base_peak_inten

In [14]:
def match_as_str(m):
    p_index, ion = m         # Unpack the peak index and the matching composition
    pm, pi = peaks[p_index]  # peak mass and intensity
    
    delta = (pm - ion.Mass) * 1000   # error in mmu
    
    return f'{p_index:5}:{pm:10.4f} ({delta:5.1f}) {pi:12.1f} {ion.Mass:10.4f}  {ion.Root:8}{ion.Name}'


## 5.2 Setup

In [15]:
save_matches = False                # do we want to save the matched peaks (as mass, inten, match name)?

include_large_unmatched = True      # do we want to include the larger unmatched peaks (by default > 1% base peak inten)

compounds_as_string = '_'.join([c[0] for c in base_compounds])


In [16]:
# Set up fie names and paths...this is platform independent (i.e. we don't need to know the separator character)
f_dir = os.sep + os.path.join('Users','ronbonner','Desktop')

peak_file = "201023 Erngren guanosine peaklist.txt"

peak_file_path = os.path.join(f_dir, peak_file)

peaks, tic, base_peak_inten = read_peak_list(peak_file_path)

print(f'{len(peaks)}, peaks read. TIC {tic}, base peak inten {base_peak_inten}')

print(peak_file_path)

1862, peaks read. TIC 266939600.0, base peak inten 30100000.0
/Users/ronbonner/Desktop/201023 Erngren guanosine peaklist.txt


## 5.3 Match ions

We first match the ions generated by the calculator. In a subsequent step we look specifically for the 13C forms of matcjed peaks.

**Note**: to repeat the math without changing the ion forms, run this cell and all below

In [17]:
ions = sorted(ion_forms, key = lambda x: x.Mass)   # sort values by mass to check for matches...

peak_index, ion_index, peaks_matched = 0, 0, 0
peak_half_window = 0.005
matches = []   # this is going to end up as a list of tuples : (peak index, matched composition)

# Loop all the values and peaks looking for matches within the specified window
while (ion_index < len(ions)) and (peak_index < len(peaks)):

    this_peak, this_ion = peaks[peak_index], ions[ion_index]
    low_peak, high_peak = this_peak[0]-peak_half_window, this_peak[0]+peak_half_window
  
    # Increment the vaue if it's too low and the peak if the value is too high
    if this_ion.Mass < low_peak:
        ion_index += 1
        continue

    if this_ion.Mass > high_peak:
        peak_index += 1
        continue

    # save peak index and ion composition
    # since there may be more than one peak that matches this ion value, we look ahead at the peaks
    # using a separate index so the current peak can be used with the next ion value
    # we also track the ions matched since some ions may have more than one matching peak

    matches.append((peak_index, this_ion))    # reference to peak and this composition
    peaks_matched += 1   
    
    look_ahead = peak_index + 1
 
    # look ahead at the peaks while they're still within the search window and add any matches to the list
    while (look_ahead < len(peaks)):
                
        look_ahead_peak = peaks[look_ahead]
        
        if(look_ahead_peak[0] - this_ion.Mass) > peak_half_window:
            break
            
        matches.append((look_ahead, this_ion))
        look_ahead +=1
        peaks_matched += 1 


    ion_index += 1 # increment ion index but not peak_index - there may be more than one ion within the window..

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

initial_matches = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(initial_matches)

99 peaks matched (42.3% tic), 99 total matches from 1862 peaks


In [18]:
# Look for C13 isotopes of matched peaks

c13_matches = []

last_matched_mass = 0
c13_half_window = 0.005
max_C13_count = 4

for peak_index in list(matched_indices):    #only need to look at each peak once
    
    m_mass, _ = peaks[peak_index]     # don't need inten
    
    next_peak_index = peak_index      #start looking at the next higher peak
        
    keep_going = True
    
    for c13_count in range(1, max_C13_count+1):  #look for 1,2,3... C13
    
        c13_mass = m_mass + (c13_count * 1.004)   #expected c13 mass
        c13_name = f'{m_mass:.4f}(+{c13_count})'
        c13_comp = Composition(c13_name, 1, c13_mass, '13C')  
        
        while next_peak_index < len(peaks) - 1:
            
            next_peak_index += 1  # point at next value in list
                
            next_peak_mass, next_peak_inten = peaks[next_peak_index]
            
            if next_peak_mass > (c13_mass + c13_half_window):
                keep_going = False       # when one isotope is not matched we abort and stop looking for more
                break
                
            if next_peak_mass > (c13_mass - c13_half_window):
                c13_matches.append((next_peak_index, c13_comp))
                break
        
        if not keep_going:
            break     # leave 13c for loop   

# for c13m in c13_matches
#     print(c13m)

matches += c13_matches

matches = sorted(matches, key = lambda x: x[0])  # sort by peak index...

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

after_13c_match = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(after_13c_match)

160 peaks matched (47.7% tic), 160 total matches from 1862 peaks


In [19]:
for m in matches:
    print(match_as_str(m))

  423:  284.0987 ( -0.2)     555000.0   284.0989  Guan    Guan.H+
  429:  285.0997 ( -3.0)      72800.0   285.1027  13C     284.0987(+1)
  434:  286.1029 ( -3.8)      24900.0   286.1067  13C     284.0987(+2)
  492:  306.0810 (  0.1)   15600000.0   306.0809  Guan    Guan.Na-H.H+
  496:  307.0837 ( -1.3)    1790000.0   307.0850  13C     306.0810(+1)
  499:  308.0857 ( -3.3)     246000.0   308.0890  13C     306.0810(+2)
  536:  322.0552 (  0.4)   30100000.0   322.0548  Guan    Guan.K-H.H+
  540:  323.0577 ( -1.5)    3320000.0   323.0592  13C     322.0552(+1)
  544:  324.0544 (  1.5)    2180000.0   324.0529  Guan    Guan.K*H.H+
  545:  325.0561 ( -2.3)     297000.0   325.0584  13C     324.0544(+1)
  550:  326.0579 ( -4.5)      44200.0   326.0624  13C     324.0544(+2)
  559:  328.0632 (  0.4)    8550000.0   328.0628  Guan    Guan.2(Na-H).H+
  564:  329.0655 ( -1.7)     982000.0   329.0672  13C     328.0632(+1)
  569:  330.0678 ( -3.4)     142000.0   330.0712  13C     328.0632(+2)
  623:  34

In [20]:
def get_unmatched_indices(matched_indices, peaks, threshold):
    
    peak_matches = [True if i in matched_indices else False for i in range(len(peaks))]
    
    unmatched = [i for i in range(len(peaks)) if not peak_matches[i]]
    
    return [i for i in unmatched if peaks[i][1] >= threshold]

In [21]:
threshold_percent = 1

bpi_percent_thresh = threshold_percent * base_peak_inten / 100

unmatched_indices = get_unmatched_indices(matched_indices, peaks, bpi_percent_thresh)  # get the unmatched peask > 1% base peak

unmatched_mass, unmatched_inten = 0,0

for pi in unmatched_indices:
    m,inten = peaks[pi]
    percent_base_peak = inten * 100/ base_peak_inten
    print(f'{m:10.4f} {inten:10.0f} {percent_base_peak:8.2f}% base peak')
    
    if inten > unmatched_inten:
        unmatched_mass, unmatched_inten = m,inten
        
largest_unmatched_string = f'Largest unmatched {unmatched_mass}, {unmatched_inten} {unmatched_inten * 100/ base_peak_inten:.1f}% base'


  102.1272    1480000     4.92% base peak
  103.9550     385000     1.28% base peak
  106.9501     387000     1.29% base peak
  118.0857     561000     1.86% base peak
  118.1218     523000     1.74% base peak
  122.9241    1880000     6.25% base peak
  135.0296    1900000     6.31% base peak
  141.1127     320000     1.06% base peak
  152.0562   18800000    62.46% base peak
  153.0578    1330000     4.42% base peak
  155.1286     883000     2.93% base peak
  156.0415     601000     2.00% base peak
  174.0380    1650000     5.48% base peak
  190.0117     351000     1.17% base peak
  190.9112     498000     1.65% base peak
  196.0197     457000     1.52% base peak
  206.8851     369000     1.23% base peak
  217.1040     351000     1.17% base peak
  230.2476     943000     3.13% base peak
  242.9249     827000     2.75% base peak
  258.8989    1370000     4.55% base peak
  267.1693     630000     2.09% base peak
  273.1669     724000     2.41% base peak
  274.8730    1210000     4.02% ba

In [22]:
if save_matches:

    out_path, _ = os.path.splitext(peak_file_path)    # path without extension

    out_path = f'{out_path} {compounds_as_string} matches.txt'
    
    to_save = matches
    
    if include_large_unmatched:
        to_save = matches + [(i, Composition('n/a',1, 0.1)) for i in unmatched_indices]  # append a list of the unmatched ions as empty matches
    
    to_save = sorted(to_save, key = lambda x: x[0])  # was x[1] peak mass, now x[0], peak index

    with open(out_path, 'w') as f:  

        for m in to_save:
            
            peak_index, matched_ion = m
            
            pm, pi = peaks[peak_index]
            
            f.write(f'{pm:.4f}\t{pi:.1f}\t{matched_ion.Name}{os.linesep}')
                    
        f.close()

    print(len(matched_indices), 'written')

    print(out_path)

In [23]:
# summarize matches

def limits_as_string(limits):
    non_zero_limits = [l for l in limits if l[1] > 0]
    if len(non_zero_limits) == 0:
        return ""
    else:
        desc = ",".join([f'{l}' for l in non_zero_limits])
        return desc
                        
if multimer_limit > 1: print(f'Up to {multimer_limit} multimers')
if include_hetero_dimers: print(f'Include heterodimers')
print(f'{ionization} mode')

desc = limits_as_string(phase1_limits)
if desc: print(f'Phase 1: {desc}')

desc = limits_as_string(phase2_limits)
if desc: print(f'Phase 2: {desc}')

desc = limits_as_string(adduct_limits)
if desc: print(f'Adducts: {desc}, max count = {max_adduct_count}')

desc = limits_as_string(loss_limits)
if desc: print(f'Losses: {desc}')  

print()
print(f'{peak_file_path} matched with half window {peak_half_window}')
print(f'Looking for <= {max_C13_count} 13C isotopes with half window {c13_half_window}')

print(initial_matches)
print('After 13C match',after_13c_match)

print(f'{len(unmatched_indices)} unmatched peaks gt {threshold_percent}%, {largest_unmatched_string}')

Up to 3 multimers
Include heterodimers
positive mode
Adducts: ('Na-H', 6),('K-H', 6),('K*H', 4),('CH2O2', 10), max count = 16

/Users/ronbonner/Desktop/201023 Erngren guanosine peaklist.txt matched with half window 0.005
Looking for <= 4 13C isotopes with half window 0.005
99 peaks matched (42.3% tic), 99 total matches from 1862 peaks
After 13C match 160 peaks matched (47.7% tic), 160 total matches from 1862 peaks
60 unmatched peaks gt 1%, Largest unmatched 152.0562, 18800000.0 62.5% base
